<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **CrewAI 101: Building Multi-Agent AI Systems**


Estimated time needed: **45** minutes


In this lab, we build a GenAI-powered content creation pipeline designed to transform raw research into polished, insightful blog posts.

We'll build a  CrewAI system which uses a sequential process where a Research Analyst agent gathers cutting-edge information from real-time tools like web search, and a Content Strategist agent who rewrites that information into clear, engaging content for a tech-savvy audience. We'll also create a workflow which demonstrates how autonomous agents can collaborate like human teams, moving from knowledge extraction to audience-ready content, without manual intervention.

This project is perfect for beginners who want to learn the fundamentals of multi-agent AI automation using CrewAI. You'll see how roles, tools, and tasks come together to create streamlined, intelligent workflows that save time and enhance content quality.


## __Table of Contents__

<ol>
    <li><a href="#Objectives">Objectives</a></li>
    <li>
        <a href="#Setup">Setup</a>
        <ol>
            <li><a href="#Installing-Required-Libraries">Installing Required Libraries</a></li>
        </ol>
    </li>
    <li><a href="#What-is-CrewAI?">What is CrewAI?</a></li>
    <li><a href="#Setting-Up-SerperDevTool">Setting Up SerperDevTool</a></li>
    <li><a href="#Setting-up-our-LLM">Setting up our LLM</a></li>
    <li><a href="#Agents-in-CrewAI">Agents in CrewAI</a></li>
    <li><a href="#Tasks-in-CrewAI">Tasks in CrewAI</a></li>
    <li><a href="#CrewAI-Workflow">CrewAI Workflow</a></li>
    
    
</ol>

<a href="#Exercises">Exercises</a>


## Objectives

After completing this lab, you will be able to:

- Leverage **CrewAI** to automate multi-agent workflows for intelligent content generation.  
- Understand the **key components of CrewAI**—agents, tasks, tools, and processes—and how they work together in a sequential pipeline.  
- Implement **real-world AI collaboration scenarios**, such as transforming technical research into reader-friendly content.    
- Develop foundational skills to **extend and scale CrewAI workflows** across various domains like marketing, education, and research automation.


## Setup


## Required Libraries

For this lab, we will be using the following Python libraries:

* [`crewai`](https://pypi.org/project/crewai/) – The core framework for building collaborative AI workflows using agents, tasks, and process management.
* [`crewai-tools`](https://pypi.org/project/crewai-tools/) – A set of prebuilt tools (like web search, file I/O, and APIs) that can be used by CrewAI agents.
* [`langchain`](https://www.langchain.com/) – Provides core utilities for working with LLMs, prompts, tools, and memory management (used under the hood by CrewAI).
* [`langchain-community`](https://pypi.org/project/langchain-community/) – Offers integration with open-source and third-party tools used in the broader LangChain ecosystem.


### Installing Required Libraries

The following required libraries are __not__ pre-installed in the Skills Network Labs environment. __You will need to run the following cell__ to install them:


In [1]:
%pip install langchain==0.3.20 | tail -n 1 
%pip install crewai==0.80.0 | tail -n 1
%pip install langchain-community==0.3.19 | tail -n 1 
%pip install crewai-tools==0.38.0 | tail -n 1
%pip install databricks-sdk==0.57.0| tail -n 1

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


----


## **What is CrewAI?**  

CrewAI is a **cutting-edge framework** that empowers us to create and manage teams of **autonomous AI agents** designed to collaborate on complex tasks. Think of it as our ultimate toolkit for assembling a team of virtual experts, where each member plays a **specific role**, uses **unique tools**, and works toward **clear goals**. These agents aren’t just working in isolation; they collaborate, communicate, and solve problems as a synchronized team, enabling us to achieve more than ever before.   

### **Why CrewAI?**  
Imagine you’re leading a project. You need specialists—each with unique expertise—who can work together to achieve a common goal. CrewAI replicates this dynamic in the world of AI by:  
- Assigning **roles** to agents based on their purpose (e.g., a planner, an executor, or a coordinator).  
- Equipping them with **tools** to perform their tasks efficiently.  
- Directing them with **goals** to ensure their efforts align with the broader mission.  

This collaborative framework ensures that your AI agents can tackle challenges that are too big or too complex for a single agent to handle. Whether it's **automation**, **decision-making**, or **simulating real-world scenarios**, CrewAI empowers you to orchestrate your AI teams like never before.  

### **How CrewAI Works**
At its core, CrewAI provides us with a high-level framework to build “crews”—groups of role-playing agents that interact and collaborate to achieve shared objectives. Each agent is:  
- **Assigned a Role:** Just like in a real team, every agent has a specialized function, whether it’s planning, executing, or coordinating tasks.  
- **Equipped with Tools:** Agents are provided with the resources they need to perform their roles effectively.  
- **Directed by Goals:** Clear objectives ensure that every agent’s efforts align with the crew’s mission.  


## Setting Up SerperDevTool

**What is Serper?**  
Serper is a real-time Google Search API that allows AI agents to access up-to-date web information—effectively connecting your workflow to the latest content on the internet.

**Why are we using Serper in our workflow?**  
Our research agent needs current, reliable information to uncover trends, breakthroughs, and insights on evolving topics like generative AI, quantum computing, or sustainability. Without web access, the agent would be limited to static, pre-trained knowledge and unable to reflect the latest developments.

To use the `SerperDevTool`, it requires an **API key**. This key grants access to the web search service and allows our agents to fetch real-time data during execution.

> You will need to obtain your API Key from [serper.dev](https://serper.dev).  
> - Sign up or log in with your email  
> - Navigate to the **Dashboard**  
> - Click on **API Keys**  
> - Copy the key and replace `API_KEY` in your code with the value provided

To learn more about the `SerperDevTool` and its capabilities, visit the [official documentation](https://serper.dev/).


Enter  API key 


In [2]:
import os 
os.environ['SERPER_API_KEY'] = "API_KEY_HERE"

Import ```SerperDevTool``` from ```crewai_tools```. 


In [3]:
%%capture

from crewai_tools import SerperDevTool

Initialize the SerperDev search tool  object (requires an API key)


In [4]:
search_tool=SerperDevTool()
print(type(search_tool))

<class 'crewai_tools.tools.serper_dev_tool.serper_dev_tool.SerperDevTool'>


Run a search query 


In [5]:
search_query = "Latest Breakthroughs in machine learning"
search_results =search_tool.run(query=search_query )

# Print the results
print(f"Search Results for '{search_results}':\n")

Using Tool: Search the internet with Serper
Search Results for '{'searchParameters': {'q': 'Latest Breakthroughs in machine learning', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': '5 Breakthrough Machine Learning Research Papers ...', 'link': 'https://machinelearningmastery.com/5-breakthrough-machine-learning-research-papers-already-in-2025/', 'snippet': "In this article, you'll learn about five significant machine learning research papers published so far in 2025.", 'position': 1}, {'title': 'Advancements in AI and Machine Learning', 'link': 'https://ep.jhu.edu/news/advancements-in-ai-and-machine-learning/', 'snippet': 'AI and ML advancements are transforming engineering by automating complex tasks and enhancing decision-making processes for professionals.', 'position': 2}, {'title': 'Latest AI Breakthroughs and News: June, July, August 2025', 'link': 'https://www.crescendo.ai/news/latest-ai-news-and-updates', 'snippet': 'Summary: Researchers have unveiled a

The ````search_results```` dictionary has a lot of info, so here is an overview of what each key contains:


- **searchParameters**: Query metadata (term, engine, result count)
- **organic**: Search results (title, link, snippet, position)
- **peopleAlsoAsk**: Related questions with answers
- **relatedSearches**: Alternative search queries
- **credits**: API usage tracking


In [6]:
print("keys of search_results", search_results.keys())

keys of search_results dict_keys(['searchParameters', 'organic', 'credits'])


## Setting up our LLM

Next, we need to set up our **LLM (Large Language Model)**—this can be **any model** based on our needs. Here, we are going to use **Meta Llama 3.3 70b instruct**. The choice of model depends on factors such as **accuracy, speed, and recipe understanding** for our meal planning tasks.


In [7]:
from crewai import LLM

llm = LLM(
        model="watsonx/meta-llama/llama-3-3-70b-instruct",
        base_url="https://us-south.ml.cloud.ibm.com",
        project_id="skills-network",
        max_tokens=2000,
)

## **Agents in CrewAI**  

In CrewAI, **agents** are the foundational units of any multi-agent system. Each agent is designed to perform a specific role, solve tasks autonomously, and collaborate seamlessly with other agents. They’re more than mere programs—they are your specialized team members in an AI-powered ecosystem.  

---




A CrewAI agent isn’t just a block of code; it’s a thoughtfully designed entity with the following parameters:  

1. **Role**  
   An agent’s role defines its purpose in the system. Roles are as diverse as your project needs, such as a **"Data Researcher"** hunting for insights or a **"Reporting Analyst"** preparing comprehensive summaries.  

2. **Goal**  
   Each agent operates with a defined goal—a guiding star that shapes its decisions and actions. For instance, an agent with the goal to **“Uncover cutting-edge developments in AI”** will consistently align its behavior to fulfill this objective.  

3. **Backstory**  
   An agent’s backstory is like its resume, providing context or personality that influences how it behaves and interacts. For example, a seasoned **“Senior Data Researcher”** with years of experience might approach tasks differently from a **“Junior Analyst”** just starting out. This feature adds depth and relatability to agent interactions, making them more dynamic and tailored.  


4. **Tools**  
   Just like any professional needs the right tools to excel, agents in CrewAI are equipped with specialized tools to boost their performance. Whether it’s a **web search utility** for gathering information, a **data analysis engine** for crunching numbers, or an **API connector** to integrate external services, tools expand an agent’s capabilities. The right tool can help an agent complete its tasks more efficiently and effectively, enabling it to work smarter, not harder.  

5. **Configuration**  
   Agents in CrewAI are configured using simple YAML files, offering a modular, readable, and scalable approach to defining their attributes. This makes setting up agents intuitive, even for large systems ( in this tutroal we will not use a YML files 




####  **Defining an Agent Directly as a Python Object**
For more flexibility or when working in a programmatic environment, you can define agents directly in your code. This approach allows you to quickly integrate dynamic parameters and logic into the agent’s setup.

In this section, we're defining the research agent which will gather and analyze information from the web. This "Senior Research Analyst" uses the SerperDevTool to search for relevant content, working independently without delegation. The agent serves as the first step in our workflow, collecting the raw data that other agents will later refine and present.

Example of defining an agent in **Python**:



In [8]:
from crewai import Agent

research_agent = Agent(
  role='Senior Research Analyst',
  goal='Uncover cutting-edge information and insights on any subject with comprehensive analysis',
  backstory="""You are an expert researcher with extensive experience in gathering, analyzing, and synthesizing information across multiple domains. 
  Your analytical skills allow you to quickly identify key trends, separate fact from opinion, and produce insightful reports on any topic. 
  You excel at finding reliable sources and extracting valuable information efficiently.""",
  verbose=True,
  allow_delegation=False,
  llm = llm,
  tools=[SerperDevTool()]
)


In this Python example, an agent is created with the same role, goal, backstory, and tools as the YAML example. However, this method allows you to easily pass in dynamic variables and parameters at runtime, making it ideal for scenarios where the agent configuration needs to change dynamically.


In [9]:
research_agent

Agent(role=Senior Research Analyst, goal=Uncover cutting-edge information and insights on any subject with comprehensive analysis, backstory=You are an expert researcher with extensive experience in gathering, analyzing, and synthesizing information across multiple domains. 
  Your analytical skills allow you to quickly identify key trends, separate fact from opinion, and produce insightful reports on any topic. 
  You excel at finding reliable sources and extracting valuable information efficiently.)

In CrewAI, we use multiple specialized agents to complete complex tasks through collaboration. In our research-report example:

1. We created a **Researcher Agent** that gathers information
2. Now we will create a **Writer Agent** that takes the output from our Researcher Agent
3. The Writer transforms research findings into well-structured content for the target audience

Let's create the writer agent with the following parameters:

* **role**: 'Tech Content Strategist' - Job function within the workflow
* **goal**: 'Craft well-structured and engaging content based on research findings' - The agent's specific objective
* **backstory**: Background that shapes the agent's approach and style
* **verbose**: True - Controls logging detail level
* **allow_delegation**: True - Enables task assignment to other agents


In [10]:
# Define your agents with roles and goals
# Define the Writer Agent
writer_agent = Agent(
  role='Tech Content Strategist',
  goal='Craft well-structured and engaging content based on research findings',
  backstory="""You are a skilled content strategist known for translating 
  complex topics into clear and compelling narratives. Your writing makes 
  information accessible and engaging for a wide audience.""",
  verbose=True,
  llm = llm,
  allow_delegation=True
)

In [11]:
writer_agent 

Agent(role=Tech Content Strategist, goal=Craft well-structured and engaging content based on research findings, backstory=You are a skilled content strategist known for translating 
  complex topics into clear and compelling narratives. Your writing makes 
  information accessible and engaging for a wide audience.)

## **Tasks in CrewAI**
Tasks are like to-do items for our AI agents. Each task has specific instructions, details, and tools for the agent to follow and complete the job.

For example:
- A task could ask an agent to "research the latest AI trends."
- Another task could ask a different agent to "write a detailed report based on the research."



Here is an outline of the porcess:

1. **Define agents** with their roles, goals, and tools
2. **Create tasks** and assign them to specific agents
3. **Combine agents and tasks** into a Crew with an execution process


#### **How Tasks Work**  
There are two ways tasks can run:  

1. **Sequential**: Tasks are executed one after the other, like following a recipe step-by-step. Each task waits for the previous one to finish.  
2. **Hierarchical**: Tasks are assigned based on agent skills or roles, and multiple tasks can run in parallel if they don’t depend on each other.  




#### **What Can a Task Include?**
Each task has these details:
- **Description**: What needs to be done.
- **Expected Output**: What the result should look like.
- **Agent**: Who’s responsible for the task.
- **Tools**: The tools the agent can use for this task.
- **Context**: Outputs from other tasks that help this task.
- **Async Execution**: Whether the task runs in the background or not.
- **Output Format**: Whether the results are plain text, JSON, or a structured model.


Here's how we set up a Crew (our team of agents) and tasks in code `research_task` and `writer_task`. 

In this step, we define a Task for the Researcher Agent. This task will involve gathering and analyzing key insights on any topic specified through the `{topic}` parameter. The agent will use the SerperDevTool to uncover major trends, identify new technologies, and evaluate their effects on the industry. This flexible approach allows us to research different subjects by simply changing the input parameter when kicking off the crew.


In [12]:
from crewai import Task

research_task = Task(
  description="Analyze the major {topic}, identifying key trends and technologies. Provide a detailed report on their potential impact.",
  agent=research_agent,
  expected_output="A detailed report on {topic}, including trends, emerging technologies, and their impact."
)

Now, we will define the task for the Writer Agent, who will take the research findings and transform them into a well-structured article. The Writer Agent will ensure the content is engaging, informative, and easy to understand, making complex topics more accessible.


In [13]:
# Create a task for the Writer Agent
writer_task = Task(
  description="Create an engaging blog post based on the research findings about {topic}. Tailor the content for a tech-savvy audience, ensuring clarity and interest.",
  agent=writer_agent,
  expected_output="A 4-paragraph blog post on {topic}, written clearly and engagingly for tech enthusiasts."
)

## CrewAI Workflow

The  `Crew` object, which is the central orchestration mechanism in CrewAI. This crew brings together our specialized agents and their assigned tasks into a cohesive workflow.

The `Crew` constructor takes several important parameters:
- `agents`: A list of the AI agents that will be part of this crew ```research_agent``` abd  ```writer_agent```
- `tasks`: A list of specific tasks these agents will perform ```research_task``` and ```writer_task```
- `process`: Defines how tasks will be executed - in this case `Process.sequential means tasks will run one after another in the specified order (research first, then writing)
- `verbose`: When set to `True`, this enables detailed logging, making it easier to follow the crew's execution and troubleshoot any issues

Once configured, you can start the entire workflow with a single command: `crew.kickoff()`, which will execute the tasks in sequence and return the final results.


In [14]:
from crewai import Crew, Process

crew = Crew(
    agents=[research_agent, writer_agent],
    tasks=[research_task, writer_task],
    process=Process.sequential,
    verbose=True 
)

The method ```kickoff()``` sets everything rolling - it starts all your agents working on their tasks and returns the results when they're done. By using ```inputs={"topic": "quantum computing breakthroughs of 2024"}```, we can specify exactly what subject our agents should research, making our system flexible enough to analyze any topic without changing the task definitions.


In [15]:
result = crew.kickoff(inputs={"topic": "Latest Generative AI breakthroughs"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: eb906def-a581-40a0-aabe-035156e76fd0                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Task: Analyze the major Latest Generative AI breakthroughs, identifying key trends and technologies. Provide   │
│  a detailed report on their potential impact.                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Thought: Thought: To analyze the major latest generative AI breakthroughs, I need to search for the most       │
│  recent and relevant information on this topic. This will involve identifying key trends, emerging              │
│  technologies, and their potential impact. I should start by searching the internet for the latest news and     │
│  developments in generative AI.                                                                                 │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"latest generative AI breakthroughs\"}"                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  The latest developments in generative AI include the development of new tools and technologies such as         │
│  Grok-Imagine, Agentic AI, Physical AI, and Sovereign AI. These advancements are transforming industries and    │
│  have the potential to impact various aspects of society.                                                       │
│                                                                                                                 │
│  Grok-Imagine is a new image and video generation tool launched by xAI, which lets users quickly create         │
│  AI-generated videos and images. Agentic AI is a type of AI that integrates reasoning, web browsing, and code   │
│  execution to autonomously perform tasks. Physical AI refers to the use of AI in physical systems, such as      │
│  robotics and autonomous vehicles. Sovereign AI is a term used to describe AI systems that are designed to      │
│  operate independently and make decisions without human intervention.                                           │
│                                                                                                                 │
│  The current state of generative AI is characterized by significant advancements in recent years, with the      │
│  technology being used in a wide range of applications, including image and video generation, natural language  │
│  processing, and game playing. However, there are also challenges associated with generative AI, such as the    │
│  potential for job displacement, bias in AI decision-making, and the need for more transparent and explainable  │
│  AI systems.                                                                                                    │
│                                                                                                                 │
│  The potential applications of generative AI are vast and varied, and include areas such as healthcare,         │
│  finance, education, and entertainment. For example, generative AI can be used to generate synthetic data for   │
│  training machine learning models, or to create personalized educational content for students. In healthcare,   │
│  generative AI can be used to analyze medical images and diagnose diseases.                                     │
│                                                                                                                 │
│  Overall, the latest developments in generative AI have the potential to transform many aspects of society,     │
│  and it is likely that we will see significant advancements in this field in the coming years.                  │
│                                                                                                                 │
│  Some of the key challenges associated with generative AI include:                                              │
│                                                                                                                 │
│  * Job displacement: Generative AI has the potential to automate many jobs, particularly those that involve     │
│  repetitive or ...                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Tech Content Strategist                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The latest generative AI breakthroughs are transforming industries and have the potential to impact various    │
│  aspects of society. One of the most significant advancements is the development of new tools and technologies  │
│  such as Grok-Imagine, Agentic AI, Physical AI, and Sovereign AI. Grok-Imagine is a powerful tool for           │
│  generating images and videos, and it has the potential to impact various industries such as entertainment,     │
│  education, and marketing. Agentic AI refers to AI systems capable of making decisions and executing tasks      │
│  independently, with minimal human oversight. Physical AI enhances operational resilience in manufacturing,     │
│  and Sovereign AI refers to AI systems that respect local privacy laws and encourage the rise of regional AI    │
│  solutions.                                                                                                     │
│                                                                                                                 │
│  The potential impact of these breakthroughs is significant, and they are expected to shape the future of AI    │
│  in 2026 and beyond. Agentic AI, Physical AI, and Sovereign AI are expected to transform industries and         │
│  revolutionize the way we live and work. They have the potential to increase productivity, improve              │
│  decision-making, and enhance operational resilience. For example, Agentic AI can be used to automate routine   │
│  tasks, freeing up human workers to focus on more complex and creative tasks. Physical AI can be used to        │
│  improve manufacturing processes, reducing waste and increasing efficiency. Sovereign AI can be used to         │
│  develop AI systems that are tailored to specific regions and cultures, promoting diversity and inclusivity in  │
│  the development of AI.                                                                                         │
│                                                                                                                 │
│  However, there are also potential risks and challenges associated with these breakthroughs, such as job        │
│  displacement, bias, and privacy concerns. Therefore, it is essential to develop and implement these            │
│  technologies responsibly and with careful consideration of their potential impact on society. This includes    │
│  ensuring that AI systems are transparent and explainable, and that they are designed to promote fairness and   │
│  equity. It also includes providing training and education to workers who may be displaced by automation, and   │
│  ensuring that the benefits of AI are shared by all. By developing and implementing generative AI technologies  │
│  responsibly, we can realize the full potential of these technologies and create a brighter future for all.     │
│                                                                                                                 │
│  In conclusion, the latest generative AI breakthroughs have the potential to revolutionize various industries   │
│  and aspects of society, and it is crucial to develop and implement these technologies responsibly and with     │
│  careful consideration of their potential impact. The future of AI is expected to be shaped by these            │
│  breakthroughs, and it is essential to stay informed an

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: ca8faca3-e18a-4490-8c71-e454f910668b                                                                     │
│  Agent: Tech Content Strategist                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: eb906def-a581-40a0-aabe-035156e76fd0                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: The latest generative AI breakthroughs are transforming industries and have the potential to     │
│  impact various aspects of society. One of the most significant advancements is the development of new tools    │
│  and technologies such as Grok-Imagine, Agentic AI, Physical AI, and Sovereign AI. Grok-Imagine is a powerful   │
│  tool for generating images and videos, and it has the potential to impact various industries such as           │
│  entertainment, education, and marketing. Agentic AI refers to AI systems capable of making decisions and       │
│  executing tasks independently, with minimal human oversight. Physical AI enhances operational resilience in    │
│  manufacturing, and Sovereign AI refers to AI systems that respect local privacy laws and encourage the rise    │
│  of regional AI solutions.                                                                                      │
│                                                                                                                 │
│  The potential impact of these breakthroughs is significant, and they are expected to shape the future of AI    │
│  in 2026 and beyond. Agentic AI, Physical AI, and Sovereign AI are expected to transform industries and         │
│  revolutionize the way we live and work. They have the potential to increase productivity, improve              │
│  decision-making, and enhance operational resilience. For example, Agentic AI can be used to automate routine   │
│  tasks, freeing up human workers to focus on more complex and creative tasks. Physical AI can be used to        │
│  improve manufacturing processes, reducing waste and increasing efficiency. Sovereign AI can be used to         │
│  develop AI systems that are tailored to specific regions and cultures, promoting diversity and inclusivity in  │
│  the development of AI.                                                                                         │
│                                                                                                                 │
│  However, there are also potential risks and challenges associated with these breakthroughs, such as job        │
│  displacement, bias, and privacy concerns. Therefore, it is essential to develop and implement these            │
│  technologies responsibly and with careful consideration of their potential impact on society. This includes    │
│  ensuring that AI systems are transparent and explainable, and that they are designed to promote fairness and   │
│  equity. It also includes providing training and education to workers who may be displaced by automation, and   │
│  ensuring that the benefits of AI are shared by all. By developing and implementing generative AI technologies  │
│  responsibly, we can realize the full potential of these technologies and create a brighter future for all.     │
│                                                                                                                 │
│  In conclusion, the latest generative AI breakthroughs have the potential to revolutionize various industries   │
│  and aspects of society, and it is crucial to develop and implement these technologies responsibly and with     │
│  careful consideration of their potential impact. The 

The result is a ```crew_output``` 


In [16]:
type(result)

crewai.crews.crew_output.CrewOutput

In [17]:
result

CrewOutput(raw='The latest generative AI breakthroughs are transforming industries and have the potential to impact various aspects of society. One of the most significant advancements is the development of new tools and technologies such as Grok-Imagine, Agentic AI, Physical AI, and Sovereign AI. Grok-Imagine is a powerful tool for generating images and videos, and it has the potential to impact various industries such as entertainment, education, and marketing. Agentic AI refers to AI systems capable of making decisions and executing tasks independently, with minimal human oversight. Physical AI enhances operational resilience in manufacturing, and Sovereign AI refers to AI systems that respect local privacy laws and encourage the rise of regional AI solutions.\n\nThe potential impact of these breakthroughs is significant, and they are expected to shape the future of AI in 2026 and beyond. Agentic AI, Physical AI, and Sovereign AI are expected to transform industries and revolutioniz

The `result.raw` output text contains the final content produced by our last agent in the workflow. We can easily access this text to see the complete results:


In [18]:
final_output = result.raw
print("Final output:", final_output)

Final output: The latest generative AI breakthroughs are transforming industries and have the potential to impact various aspects of society. One of the most significant advancements is the development of new tools and technologies such as Grok-Imagine, Agentic AI, Physical AI, and Sovereign AI. Grok-Imagine is a powerful tool for generating images and videos, and it has the potential to impact various industries such as entertainment, education, and marketing. Agentic AI refers to AI systems capable of making decisions and executing tasks independently, with minimal human oversight. Physical AI enhances operational resilience in manufacturing, and Sovereign AI refers to AI systems that respect local privacy laws and encourage the rise of regional AI solutions.

The potential impact of these breakthroughs is significant, and they are expected to shape the future of AI in 2026 and beyond. Agentic AI, Physical AI, and Sovereign AI are expected to transform industries and revolutionize th

The `tasks_output` list gives us access to outputs from each task in the order they were executed:


In [19]:
tasks_outputs = result.tasks_output

We see the output of the research task object. This lets us access both the task description and the content the agent produced:


In [20]:
print("Task Description", tasks_outputs[0].description)
print("Output of research task ",tasks_outputs[0])

Task Description Analyze the major Latest Generative AI breakthroughs, identifying key trends and technologies. Provide a detailed report on their potential impact.
Output of research task  The latest generative AI breakthroughs include the development of new tools and technologies such as Grok-Imagine, Agentic AI, Physical AI, and Sovereign AI. These advancements are transforming industries and have the potential to impact various aspects of society.

Grok-Imagine is a powerful tool for generating images and videos, and it has the potential to impact various industries such as entertainment, education, and marketing. Agentic AI refers to AI systems capable of making decisions and executing tasks independently, with minimal human oversight. Physical AI enhances operational resilience in manufacturing, and Sovereign AI refers to AI systems that respect local privacy laws and encourage the rise of regional AI solutions.

The potential impact of these breakthroughs is significant, and the

We also have the description and output for the writer task using the raw property:


In [21]:
print("Writer task description:", tasks_outputs[1].description)
print(" \nOutput of writer task:", tasks_outputs[1].raw)

Writer task description: Create an engaging blog post based on the research findings about Latest Generative AI breakthroughs. Tailor the content for a tech-savvy audience, ensuring clarity and interest.
 
Output of writer task: The latest generative AI breakthroughs are transforming industries and have the potential to impact various aspects of society. One of the most significant advancements is the development of new tools and technologies such as Grok-Imagine, Agentic AI, Physical AI, and Sovereign AI. Grok-Imagine is a powerful tool for generating images and videos, and it has the potential to impact various industries such as entertainment, education, and marketing. Agentic AI refers to AI systems capable of making decisions and executing tasks independently, with minimal human oversight. Physical AI enhances operational resilience in manufacturing, and Sovereign AI refers to AI systems that respect local privacy laws and encourage the rise of regional AI solutions.

The potentia


In addition to the task output, we can access the agent that performed each task:


In [22]:
print("We can get the agent for researcher task:  ",tasks_outputs[0].agent)
print("We can get the agent for the writer task: ",tasks_outputs[1].agent)

We can get the agent for researcher task:   Senior Research Analyst
We can get the agent for the writer task:  Tech Content Strategist


---
After your agents complete their tasks, CrewAI provides detailed performance metrics that help you monitor resource usage and optimize your multi-agent systems. Token usage analytics are particularly important as they directly impact operational costs and system efficiency.


In [23]:
token_count = result.token_usage.total_tokens
prompt_tokens = result.token_usage.prompt_tokens
completion_tokens = result.token_usage.completion_tokens

print(f"Total tokens used: {token_count}")
print(f"Prompt tokens: {prompt_tokens} (used for instructions to the model)")
print(f"Completion tokens: {completion_tokens} (generated in response)")

Total tokens used: 28921
Prompt tokens: 23864 (used for instructions to the model)
Completion tokens: 5057 (generated in response)


## Exercises 
In these exercises, you will create a web publishing component for your fact-checking application by implementing a web designer agent and task. This final piece will transform the analyzed and written content into a professional webpage that presents verification results clearly to users.


### Exercise 1: Create a Social Media Strategist Agent

Create a Social Media Agent which curates a summary and a short-form version (such as tweets or LinkedIn posts).


In [27]:
#TODO

social_agent = Agent(
    role='Social Media Strategist',
    goal='Generate engaging social media snippets based on the full article',
    backstory="A digital storyteller who excels at crafting compelling posts to drive engagement and traffic.",
    verbose=True
)

<details>
    <summary>Click here for the solution</summary>

```python

social_agent = Agent(
    role='Social Media Strategist',
    goal='Generate engaging social media snippets based on the full article',
    backstory="A digital storyteller who excels at crafting compelling posts to drive engagement and traffic.",
    verbose=True
)


```

</details>


### Exercise 2: Defining a Social Media Strategy Task

Create a task for the Social Media Strategist agent to generate engaging and platform-specific posts (such as LinkedIn or X/Twitter) based on the research and blog content. This agent will help amplify the reach of your content by distilling key insights into short, compelling messages.


In [30]:
#TODO

social_task = Task(
    description=(
        "Summarize the blog post about {topic} into 2–3 engaging social media posts "
        "suitable for platforms like LinkedIn or Twitter. Make sure the tone is informative, "
        "professional, and encourages further reading."
    ),
    agent=social_agent,
    expected_output="A series of 2–3 well-written social posts highlighting the key insights from the blog content."
)

<details>
    <summary>Click here for the solution</summary>

```python
social_task = Task(
    description=(
        "Summarize the blog post about {topic} into 2–3 engaging social media posts "
        "suitable for platforms like LinkedIn or Twitter. Make sure the tone is informative, "
        "professional, and encourages further reading."
    ),
    agent=social_agent,
    expected_output="A series of 2–3 well-written social posts highlighting the key insights from the blog content."
)
```

</details>


### Exercise 3: Create a Complete Crew Object 

Include research, writing, and social media agents along with their tasks, configured for sequential processing with verbose output and apply the method ```kickoff()``` method.


In [31]:
#TODO

crew = Crew(
    agents=[research_agent, writer_agent, social_agent],
    tasks=[research_task, writer_task, social_task],
    process=Process.sequential,  # Tasks will be executed one after another
    verbose=True
)

# Run the crew and capture the final output (includes research, blog post, and social media content)
result = crew.kickoff(inputs={"topic": "Latest Generative AI breakthroughs"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: acb9b898-64d0-4e95-9d95-a442a0f80fff                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Task: Analyze the major Latest Generative AI breakthroughs, identifying key trends and technologies. Provide   │
│  a detailed report on their potential impact.                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Thought: Thought: To analyze the major latest Generative AI breakthroughs, I need to search for the most       │
│  recent and relevant information on the internet. This will help me identify key trends, emerging               │
│  technologies, and their potential impact.                                                                      │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"latest Generative AI breakthroughs\"}"                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Social Media Strategist                                                                                 │
│                                                                                                                 │
│  Task: Summarize the blog post about Latest Generative AI breakthroughs into 2–3 engaging social media posts    │
│  suitable for platforms like LinkedIn or Twitter. Make sure the tone is informative, professional, and          │
│  encourages further reading.                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Social Media Strategist                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  🌟 **Harnessing the Future: Generative AI Breakthroughs** 🌟                                                   │
│  Discover how the latest advancements in Generative AI are shaping industries by introducing **Agentic**,       │
│  **Physical**, and **Sovereign AI**. These transformative technologies are not just enhancing efficiency in     │
│  sectors like healthcare and finance but are also paving the way for autonomous systems and hyper-personalized  │
│  experiences. Dive deeper into how these trends are redefining our world! [Read More](#)                        │
│                                                                                                                 │
│  🤖 **Conversational AI: The Human Touch for Machines**                                                         │
│  The rise of **Conversational AI** is revolutionizing how we interact with technology. From improved customer   │
│  support to more intuitive user experiences, this technology is making interactions smoother and more natural.  │
│  Curious about the potential benefits and challenges? Explore the intricate layers of Generative AI with us!    │
│  [Read More](#)                                                                                                 │
│                                                                                                                 │
│  🔬 **Accelerating Innovation with Generative AI**                                                              │
│  Generative AI isn’t just a trend — it’s transforming scientific research and innovation! With technologies     │
│  like **Quantum Machine Learning** and **Federated Learning**, we are on the brink of unprecedented             │
│  discoveries. However, we must remain vigilant about ethical implications, including job displacement and       │
│  bias. Learn how to navigate this exciting landscape responsibly! [Read More](#)                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 8a73bd2a-d718-4a78-9903-356edd5731ed                                                                     │
│  Agent: Social Media Strategist                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: acb9b898-64d0-4e95-9d95-a442a0f80fff                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: 🌟 **Harnessing the Future: Generative AI Breakthroughs** 🌟                                     │
│  Discover how the latest advancements in Generative AI are shaping industries by introducing **Agentic**,       │
│  **Physical**, and **Sovereign AI**. These transformative technologies are not just enhancing efficiency in     │
│  sectors like healthcare and finance but are also paving the way for autonomous systems and hyper-personalized  │
│  experiences. Dive deeper into how these trends are redefining our world! [Read More](#)                        │
│                                                                                                                 │
│  🤖 **Conversational AI: The Human Touch for Machines**                                                         │
│  The rise of **Conversational AI** is revolutionizing how we interact with technology. From improved customer   │
│  support to more intuitive user experiences, this technology is making interactions smoother and more natural.  │
│  Curious about the potential benefits and challenges? Explore the intricate layers of Generative AI with us!    │
│  [Read More](#)                                                                                                 │
│                                                                                                                 │
│  🔬 **Accelerating Innovation with Generative AI**                                                              │
│  Generative AI isn’t just a trend — it’s transforming scientific research and innovation! With technologies     │
│  like **Quantum Machine Learning** and **Federated Learning**, we are on the brink of unprecedented             │
│  discoveries. However, we must remain vigilant about ethical implications, including job displacement and       │
│  bias. Learn how to navigate this exciting landscape responsibly! [Read More](#)                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

<details>
    <summary>Click here for the solution</summary>

```python
crew = Crew(
    agents=[research_agent, writer_agent, social_agent],
    tasks=[research_task, writer_task, social_task],
    process=Process.sequential,  # Tasks will be executed one after another
    verbose=True
)

# Run the crew and capture the final output (includes research, blog post, and social media content)
result = crew.kickoff(inputs={"topic": "Latest Generative AI breakthroughs"})
```

</details>


In [32]:
type(result)

crewai.crews.crew_output.CrewOutput

In [33]:
result

CrewOutput(raw='🌟 **Harnessing the Future: Generative AI Breakthroughs** 🌟  \nDiscover how the latest advancements in Generative AI are shaping industries by introducing **Agentic**, **Physical**, and **Sovereign AI**. These transformative technologies are not just enhancing efficiency in sectors like healthcare and finance but are also paving the way for autonomous systems and hyper-personalized experiences. Dive deeper into how these trends are redefining our world! [Read More](#)\n\n🤖 **Conversational AI: The Human Touch for Machines**  \nThe rise of **Conversational AI** is revolutionizing how we interact with technology. From improved customer support to more intuitive user experiences, this technology is making interactions smoother and more natural. Curious about the potential benefits and challenges? Explore the intricate layers of Generative AI with us! [Read More](#)\n\n🔬 **Accelerating Innovation with Generative AI**  \nGenerative AI isn’t just a trend — it’s transforming sc

In [34]:
final_output = result.raw
print("Final output:", final_output)

Final output: 🌟 **Harnessing the Future: Generative AI Breakthroughs** 🌟  
Discover how the latest advancements in Generative AI are shaping industries by introducing **Agentic**, **Physical**, and **Sovereign AI**. These transformative technologies are not just enhancing efficiency in sectors like healthcare and finance but are also paving the way for autonomous systems and hyper-personalized experiences. Dive deeper into how these trends are redefining our world! [Read More](#)

🤖 **Conversational AI: The Human Touch for Machines**  
The rise of **Conversational AI** is revolutionizing how we interact with technology. From improved customer support to more intuitive user experiences, this technology is making interactions smoother and more natural. Curious about the potential benefits and challenges? Explore the intricate layers of Generative AI with us! [Read More](#)

🔬 **Accelerating Innovation with Generative AI**  
Generative AI isn’t just a trend — it’s transforming scientific 

In [36]:
from IPython.display import display, Markdown

display(Markdown(f"**Final output:**\n{final_output}"))

**Final output:**
🌟 **Harnessing the Future: Generative AI Breakthroughs** 🌟  
Discover how the latest advancements in Generative AI are shaping industries by introducing **Agentic**, **Physical**, and **Sovereign AI**. These transformative technologies are not just enhancing efficiency in sectors like healthcare and finance but are also paving the way for autonomous systems and hyper-personalized experiences. Dive deeper into how these trends are redefining our world! [Read More](#)

🤖 **Conversational AI: The Human Touch for Machines**  
The rise of **Conversational AI** is revolutionizing how we interact with technology. From improved customer support to more intuitive user experiences, this technology is making interactions smoother and more natural. Curious about the potential benefits and challenges? Explore the intricate layers of Generative AI with us! [Read More](#)

🔬 **Accelerating Innovation with Generative AI**  
Generative AI isn’t just a trend — it’s transforming scientific research and innovation! With technologies like **Quantum Machine Learning** and **Federated Learning**, we are on the brink of unprecedented discoveries. However, we must remain vigilant about ethical implications, including job displacement and bias. Learn how to navigate this exciting landscape responsibly! [Read More](#)

In [37]:
tasks_outputs = result.tasks_output

In [42]:
# print("Task Description", tasks_outputs[0].description)
# print("Output of research task ",tasks_outputs[0])


display(Markdown(f"### 📝 Task Description\n{tasks_outputs[0].description}"))

display(Markdown(f"### 🔍 Output of Research Task \n {str(tasks_outputs[0])}"))


### 📝 Task Description
Analyze the major Latest Generative AI breakthroughs, identifying key trends and technologies. Provide a detailed report on their potential impact.

### 🔍 Output of Research Task 
 The latest Generative AI breakthroughs have led to significant advancements in various fields, including marketing, sales, and scientific research. Some of the key trends and emerging technologies in Generative AI include:

1. **Agentic AI**: This trend refers to the use of AI to create autonomous agents that can perform tasks on their own.
2. **Physical AI**: This trend involves the use of AI to control and interact with physical objects and environments.
3. **Sovereign AI**: This trend refers to the use of AI to create autonomous systems that can make decisions and take actions without human intervention.
4. **Hyper-Personalization**: This trend involves the use of Generative AI to create personalized content and experiences for individuals.
5. **Conversational AI**: This trend refers to the use of AI to create conversational interfaces that can interact with humans in a natural and intuitive way.
6. **Generative AI for Scientific Research**: This trend involves the use of Generative AI to accelerate scientific discovery and innovation.
7. **Human in the Generative AI Loop**: This trend refers to the use of human feedback and oversight to improve the performance and accuracy of Generative AI models.

Some of the emerging technologies in Generative AI include:

1. **Large Language Models (LLMs)**: These models are capable of processing and generating human-like language and are being used in a variety of applications, including chatbots and language translation.
2. **Explainable AI (XAI)**: This technology involves the use of AI to explain and interpret the decisions and actions of other AI systems.
3. **Edge AI**: This trend refers to the use of AI to process and analyze data at the edge of the network, reducing latency and improving real-time decision-making.
4. **Quantum Machine Learning**: This technology involves the use of quantum computing to improve the performance and efficiency of machine learning algorithms.
5. **Federated Learning**: This trend refers to the use of AI to enable multiple devices and systems to learn from each other and improve their performance over time.

Overall, the latest Generative AI breakthroughs have the potential to transform a wide range of industries and applications, from marketing and sales to scientific research and healthcare. As these technologies continue to evolve and improve, we can expect to see significant advancements in areas such as personalization, automation, and decision-making. 

The potential impact of these trends and technologies is significant, and they are expected to have a major influence on the future of AI and its applications. Some of the potential benefits of these trends and technologies include:

* Improved efficiency and productivity
* Enhanced customer experiences
* Accelerated scientific discovery and innovation
* Increased accuracy and precision
* Improved decision-making and problem-solving

However, there are also potential challenges and risks associated with these trends and technologies, including:

* Job displacement and automation
* Bias and discrimination
* Privacy and security concerns
* Dependence on technology
* Unintended consequences and errors.

Overall, the latest Generative AI breakthroughs have the potential to bring about significant benefits and improvements, but it is important to carefully consider the potential challenges and risks and to develop strategies for mitigating them.

In [44]:
display(Markdown(f"### Writer task description \n{tasks_outputs[1].description}"))

display(Markdown(f"### Output of writer task \n { tasks_outputs[1].raw}"))

### Writer task description 
Create an engaging blog post based on the research findings about Latest Generative AI breakthroughs. Tailor the content for a tech-savvy audience, ensuring clarity and interest.

### Output of writer task 
 Generative AI is revolutionizing various industries, from healthcare to finance, by accelerating tasks, improving efficiency, and enhancing decision-making. This blog post will delve into the latest trends and emerging technologies in Generative AI, including Agentic AI, Physical AI, Sovereign AI, Hyper-Personalization, Conversational AI, Generative AI for Scientific Research, and Human in the Generative AI Loop. We will also explore the potential benefits and challenges of these trends and technologies.

Latest Trends in Generative AI

1. **Agentic AI**: Agentic AI refers to AI systems that can perform tasks autonomously, making decisions and taking actions without human intervention. This technology has the potential to transform industries such as healthcare, finance, and transportation.
2. **Physical AI**: Physical AI involves the use of AI in physical systems, such as robots and drones, to perform tasks that require physical interaction with the environment. This technology has applications in areas such as manufacturing, logistics, and construction.
3. **Sovereign AI**: Sovereign AI refers to AI systems that are designed to operate independently, making decisions and taking actions without human oversight. This technology has the potential to transform industries such as healthcare, finance, and education.
4. **Hyper-Personalization**: Hyper-Personalization involves the use of AI to create personalized experiences for individuals, tailored to their specific needs and preferences. This technology has applications in areas such as marketing, advertising, and customer service.
5. **Conversational AI**: Conversational AI involves the use of AI to enable humans to interact with machines using natural language, such as voice or text. This technology has applications in areas such as customer service, tech support, and language translation.

Emerging Technologies in Generative AI

1. **Large Language Models**: Large Language Models are AI systems that are trained on vast amounts of text data, enabling them to generate human-like language and perform tasks such as language translation and text summarization.
2. **Explainable AI**: Explainable AI involves the use of techniques such as transparency and interpretability to explain the decisions and actions of AI systems. This technology has applications in areas such as healthcare, finance, and education.
3. **Edge AI**: Edge AI involves the use of AI in edge devices, such as smartphones and smart home devices, to perform tasks such as image recognition and natural language processing.
4. **Quantum Machine Learning**: Quantum Machine Learning involves the use of quantum computing to perform machine learning tasks, such as pattern recognition and predictive modeling. This technology has the potential to transform industries such as healthcare, finance, and materials science.
5. **Federated Learning**: Federated Learning involves the use of machine learning algorithms to train models on decentralized data, enabling the creation of personalized models for individual users. This technology has applications in areas such as healthcare, finance, and marketing.

Benefits and Challenges of Generative AI

The benefits of Generative AI include:

* **Increased efficiency**: Generative AI can automate tasks, freeing up humans to focus on higher-level tasks.
* **Improved decision-making**: Generative AI can provide insights and recommendations, enabling humans to make better decisions.
* **Enhanced customer experience**: Generative AI can create personalized experiences for individuals, tailored to their specific needs and preferences.

However, Generative AI also poses challenges, such as:

* **Job displacement**: Generative AI has the potential to displace human jobs, particularly in areas such as customer service and data entry.
* **Bias and ethics**: Generative AI systems can perpetuate biases and discriminate against certain groups, highlighting the need for transparency and accountability.
* **Security and privacy**: Generative AI systems can pose security and privacy risks, particularly in areas such as healthcare and finance.

Conclusion

Generative AI is a rapidly evolving field, with the potential to transform various industries and aspects of our lives. While it poses challenges, such as job displacement and bias, it also offers benefits, such as increased efficiency and improved decision-making. As we move forward, it is essential to prioritize transparency, accountability, and ethics in the development and deployment of Generative AI systems. By doing so, we can harness the potential of Generative AI to create a better future for all.

In [47]:
print("We can get the agent for researcher task:  ",tasks_outputs[0].agent)
print("We can get the agent for the writer task: ",tasks_outputs[1].agent)

We can get the agent for researcher task:   Senior Research Analyst
We can get the agent for the writer task:  Tech Content Strategist


In [48]:
token_count = result.token_usage.total_tokens
prompt_tokens = result.token_usage.prompt_tokens
completion_tokens = result.token_usage.completion_tokens

print(f"Total tokens used: {token_count}")
print(f"Prompt tokens: {prompt_tokens} (used for instructions to the model)")
print(f"Completion tokens: {completion_tokens} (generated in response)")

Total tokens used: 62079
Prompt tokens: 52383 (used for instructions to the model)
Completion tokens: 9696 (generated in response)


## Authors


[Karan Goswami](https://author.skills.network/instructors/karan_goswami)

[Kunal Makwana](https://author.skills.network/instructors/kunal_makwana)


## Change Log

<details>
    <summary>Click here for the changelog</summary>

|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2025-07-17|0.1|Karan Goswami|Initial version created|
|2025-07-22|0.2|Steve Ryan|ID review|

</details>

---


Copyright © IBM Corporation. All rights reserved.
